In [ ]:
from src.v4.torchdata import load_file, process_expression, process_json, load_vals, generate_optim_functions
from src.v4.torchengine import AnalyticalSetSympy, Function, EliminateAnalysis, EliminateAnalysisMergeResiduals
from src.v4.torchdata import print_formatted_table, fmt
from scipy.optimize import minimize
import sympy as sp
import torch
import numpy as np
np.set_printoptions(formatter={'float': lambda x: "{:0.2f}".format(x).rstrip('0').rstrip('.')})

In [2]:
symb_str_mapping = {}
all_analyses = {}
disciplines = ['pearl_geom','pearl_hydro', 
               'pearl_mass','pearl_prop',
               'pearl_comms','pearl_power', 
               'pearl_solar', 'pearl_battery',
               'pearl_prob']
flattened_output = []
equality_constraints_sympy = []
inequality_constraints_sympy = []
for file_name in disciplines:
    data = load_file(file_name)
    equality_constraints_sympy += [
        process_expression(elt, symb_str_mapping) 
        for elt in data.get('equality_constraints',[])]
    inequality_constraints_sympy += [
        process_expression(elt, symb_str_mapping) 
        for elt in data.get('inequality_constraints',[])]
    objective = data.get('objective',None)
    if objective is not None:
        objective = process_expression(objective, symb_str_mapping)
    functional_sets = data.get('functional_sets',[])
    flattened_output += functional_sets
    analysismap, symb_str_mapping = process_json(
        functional_sets, symb_str_mapping)
    all_analyses[file_name] = analysismap

In [3]:
idxrev = {i: elt for i, elt in 
            enumerate(symb_str_mapping.values())}
indices = {elt: torch.tensor([int(i)]) for i, elt in 
            idxrev.items()}

In [4]:
sets ={}
for file_name, analysismap in all_analyses.items():
        sets[file_name] = {
                idx:AnalyticalSetSympy(analysis, 
                outputvar=outputvar, indices=indices) 
                for idx,(analysis,outputvar,residual) in enumerate(analysismap)
                }
equality_constraints = EliminateAnalysisMergeResiduals(functions=[Function((
        sorted(expr.free_symbols, key=lambda s: s.name),
        sp.lambdify(sorted(expr.free_symbols, key=lambda s: s.name), expr, torch),  
        ), indices=indices) 
        for expr in equality_constraints_sympy])
inequality_constraints = EliminateAnalysisMergeResiduals(functions=[Function((
        sorted(expr.free_symbols, key=lambda s: s.name),
        sp.lambdify(sorted(expr.free_symbols, key=lambda s: s.name), expr, torch),  
        ), indices=indices) 
        for expr in inequality_constraints_sympy])
objective = Function((
        sorted(objective.free_symbols, key=lambda s: s.name),
        sp.lambdify(sorted(objective.free_symbols, key=lambda s: s.name), objective, torch),  
        ), indices=indices)

In [6]:
custom_partition_order = []
flat_sets = {}
for file_name, subsets in sets.items():
    custom_partition_order.append(())
    for elt in subsets.values():
        flatidx = len(flat_sets)
        flat_sets[flatidx] = elt
        custom_partition_order[-1] += (flatidx,)

In [10]:
sequential = {}
for partidx, elt in enumerate(custom_partition_order):
    if len(elt) ==1:
        sequential[partidx] = flat_sets[next(iter(elt))].analysis
    else:
        sequential[partidx] = EliminateAnalysis([flat_sets[idx].analysis for idx in elt], [])

In [11]:
all_res = EliminateAnalysisMergeResiduals(functions=[s.residual for s in flat_sets.values()]+[equality_constraints])

In [12]:
aao_obj = EliminateAnalysis(functions=[
    objective, inequality_constraints, all_res,
    Function(((), lambda : torch.tensor([])), indices=indices)])

In [13]:
x0 = load_vals('pearl_params', indices=indices)

In [14]:
test_idx = 78
idxrev[test_idx], x0[test_idx].item()

(eta_s, 0.27)

In [15]:
aao_obj(x0)

[tensor([1.], dtype=torch.float64),
 tensor([-8.8818e-16,  1.1111e-01,  7.7778e-01, -1.0000e+00, -4.0000e+00],
        dtype=torch.float64),
 tensor([-2.0000e-01,  7.3105e+00,  9.7990e+03,  0.0000e+00,  5.5814e-01,
          5.8333e-01, -2.1460e-01,  0.0000e+00,  0.0000e+00,  3.0787e+04,
          2.9601e+03, -7.5000e-01, -8.9796e-01,  9.0000e+00, -1.0200e+02,
         -2.1590e+00,  1.5570e+01, -2.6667e-01, -8.6364e-01, -8.6014e-01,
         -5.1145e-01,  1.5148e+02,  2.8308e+06, -9.9367e-01, -1.0000e+00,
          3.5990e+03,  4.3200e+06,  3.5990e+03,  1.8990e+03, -9.9998e-01,
          3.0000e+00,  0.0000e+00, -9.9998e-01, -9.9056e-01,  6.8067e-01,
          4.0000e+00,  1.9845e+00], dtype=torch.float64),
 tensor([])]

In [16]:
discipline_idxs = [s.analysis.structure[1][0].item() for s in flat_sets.values()]
design_idxs = [1,4,5,7,8,15]

In [17]:
np.array([idxrev[elt] for elt in discipline_idxs])

array([d, V_d, F_B, F_W, K_B, K_G, I, B_M, G_M, C_33, A_33, \omega_0,
       m_platform, m_solar, m_struct, t_d, S_w, P_move, l, L_pt, G_t, G_r,
       S, L_s, P_comms, E_move, E_hotel, E_comms, E_service, P_service,
       E_required, E_recharge, P_recharge, A_solar, C, m_batt],
      dtype=object)

In [18]:
np.array([idxrev[elt] for elt in design_idxs])

array([D_f, D_d, D_s, t_d, t_s, t_f], dtype=object)

In [19]:
numerical_values = {'D_f': 3.7, 'D_d':1., 'D_s':0.15, 't_d':0.2, 'h_f':0.09, 't_s':0.2}
bounds = {
    'D_f': (0.1,10),
    'D_d': (0.1,10),
    'D_s': (0.1,10),
    't_s': (0.1,10),
    't_d': (0.1,10),
    't_f': (0.1,10),
}
x0 = load_vals('pearl_params', indices=indices)
x0.requires_grad_(False)
for var, val in numerical_values.items():
    x0[indices[sp.symbols(var)]] = val
x0.requires_grad_(True)
solver_indices = design_idxs + discipline_idxs
# MAKE SURE TO SET INEQUALITY DIRECTION, it is different for scipy and ipopt
xguess, obj_function, ineq_function, eq_function, dobj, dineq, deq = generate_optim_functions(aao_obj, solver_indices, x0, 
                                                                                              inequality_direction='negative-null')
constraints = [{'type': 'eq', 'fun': eq_function, 'jac': deq}]
constraints.append({'type': 'ineq', 'fun': ineq_function, 'jac': dineq})
bnds_problem = [bounds.get(str(idxrev[elt]), (0, None)) for elt in solver_indices]

In [20]:
ineq_function(xguess)

tensor([-0.8000, -0.9550, -0.8333, -1.0000, -1.0000], dtype=torch.float64)

In [21]:
print_formatted_table([x0], indices, idxrev, torch.tensor(solver_indices))

D_f D_d D_s  t_d t_s t_f d   V_d F_B F_W K_B K_G I B_M G_M C_33 A_33 \omega_0 m_platform m_solar m_struct t_d S_w P_move l L_pt G_t G_r S L_s P_comms E_move E_hotel E_comms E_service P_service E_required E_recharge P_recharge A_solar C m_batt
3.7 1   0.15 0.2 0.2 0.5 0.3 1   1   1   1   1   1 1   1   1    1    1        1          1       1        0.2 1.1 1      1 1    1   1   1 1   1       1      1       1       1         1         1          1          1          1       1 1     


In [22]:
# minimize(obj_function, xguess, bounds=bnds_problem, jac=dobj, 
#          constraints=constraints, method='SLSQP', options={'maxiter':1000})

In [23]:
ineqlen = len(inequality_constraints_sympy)
eqlen = len(equality_constraints_sympy)+len(flat_sets)

In [24]:
def all_constraints(x):
    return np.concatenate([ineq_function(x), eq_function(x)])

def all_constraints_jac(x):
    return np.concatenate([dineq(x), deq(x)], axis=0)

In [25]:
from collections import namedtuple
import cyipopt
OptProblem = namedtuple('OptProblem', ['objective', 'constraints', 'gradient', 'jacobian', 'intermediate'])

lb,ub = zip(*bnds_problem)
cl = np.concatenate([-np.inf*np.ones(ineqlen), np.zeros(eqlen)])
cu = np.concatenate([np.zeros(ineqlen), np.zeros(eqlen)])

storeiter = [0]

def logiter(alg_mod, iter_count, obj_value, inf_pr, inf_du, mu, d_norm, regularization_size, alpha_du, alpha_pr, ls_trials):
    storeiter[0] = iter_count

# define the problem
probinfo = OptProblem(obj_function, all_constraints, dobj, all_constraints_jac, logiter)

prob = cyipopt.Problem(n=len(xguess), m=len(cu), lb=lb, ub=ub, cl=cl, cu=cu, 
                       problem_obj=probinfo)

In [26]:
xsol, optinfo = prob.solve(xguess)

In [27]:
optinfo

{'x': array([1.31, 1.31, 1.18, 0.1, 1.02, 0.1, 0.07, 1.37, 13417.55, 13417.55,
        0.6, 0.61, 0.14, 0.11, 0.1, 13221.28, 1013.21, 2.77, 1369.14,
        12.82, 1237.63, 0.1, 6.42, 4.28, 0.14, 0.14, 0.18, 8200.06,
        2830830.27, 0, 424.19, 15403.24, 4320000, 1527092.09, 1900, 0.04,
        5864395.32, 5864395.32, 135.75, 1.28, 9856126.59, 18.69]),
 'g': array([0, -0, -0, -4.28, -0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0, 0, 0,
        -0, 0, 0, 0, 0, 0, 0, 0, -0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        -0]),
 'obj_val': 1369.1380300413548,
 'mult_g': array([1.76, -0, -0, 0, 5.06, 8831.57, -10.92, -0, 0.05, 0, 0, 0, 0, 0,
        -0, -0.01, 0, 0.97, 0.02, -0.02, -67.87, -0.63, -0.94, -4244.12,
        2068.97, 1650.41, 0.04, -0, 19692771025473961984, -0.68, -0, -0,
        -0, -0, -0, -0, -0, 2.1, 222.88, -0, 0.02, -285.35]),
 'mult_x_L': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [28]:
all_res(x0).detach().numpy()

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0, 0, 0, -0, 0, 0, 0, 0, 0, 0,
       0, -0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0])

In [29]:
print_formatted_table([x0], 
                       indices,idxrev, 
                      torch.tensor(design_idxs))

D_f   D_d   D_s   t_d t_s   t_f
1.311 1.311 1.179 0.1 1.018 0.1


In [27]:
print_formatted_table([x0],indices,idxrev, 
                      torch.tensor(discipline_idxs+[6]))

d     V_d   F_B    F_W    K_B   K_G   I     B_M   G_M C_33   A_33     \omega_0 m_platform m_solar m_struct t_d S_w   P_move l     L_pt G_t   G_r     S      L_s      P_comms E_move E_hotel E_comms E_service P_service E_required E_recharge P_recharge A_solar C      m_batt h_f 
0.066 1.369 1.34e4 1.34e4 0.604 0.609 0.145 0.106 0.1 1.32e4 1013.206 2.775    1369.138   12.817  1237.632 0.1 6.418 4.279  0.136 0.14 0.175 8200.06 2.83e6 1.47e-17 424.192 1.54e4 4.32e6  1.53e6  1900      0.044     5.86e6     5.86e6     135.75     1.282   9.86e6 18.689 0.09


In [28]:
disciplineidx = 3
print_formatted_table([x0, sequential[disciplineidx](x0)], 
                      indices, 
                      idxrev,
                      torch.cat(sequential[disciplineidx].structure))

D_f   D_d   D_s   h_f  t_d t_s   rho_w C_d eta_m v   S_w   P_move
1.311 1.311 1.179 0.09 0.1 1.018 1000  1   0.75  0.1 6.418 4.279 
1.311 1.311 1.179 0.09 0.1 1.018 1000  1   0.75  0.1 6.418 4.279 


In [30]:
flat_sets[16].residualexpr

pi*D_s*t_s - S_w + pi*(D_d**2/2 + D_d*t_d - D_s**2/4) + pi*(D_f**2/4 + D_f*h_f - D_s**2/4)

In [31]:
sets['pearl_prop'][0].residualexpr

pi*D_s*t_s - S_w + pi*(D_d**2/2 + D_d*t_d - D_s**2/4) + pi*(D_f**2/4 + D_f*h_f - D_s**2/4)

In [37]:
sets['pearl_prop'][0].expression.subs({symbol: x0[idx] for symbol, idx in indices.items() 
                                       if idx in torch.cat(sequential[disciplineidx].structure)}).evalf()

6.41801544648614